## Feature Engineering + EDA - Flight Price Prediction 

In [3]:
## Dataset: https://github.com/krishnaik06/5-Days-Live-EDA-and-Feature-Engineering/tree/main/Flight%20Prediction

### Import the libraries

In [6]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [7]:
#import the train data
df_train=pd.read_excel("flight_prediction_Train.xlsx")

In [8]:
df_train.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302


In [9]:
# import the test data
df_test=pd.read_excel("flight_prediction_Test_set.xlsx")


In [10]:
df_test.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info
0,Jet Airways,6/06/2019,Delhi,Cochin,DEL → BOM → COK,17:30,04:25 07 Jun,10h 55m,1 stop,No info
1,IndiGo,12/05/2019,Kolkata,Banglore,CCU → MAA → BLR,06:20,10:20,4h,1 stop,No info
2,Jet Airways,21/05/2019,Delhi,Cochin,DEL → BOM → COK,19:15,19:00 22 May,23h 45m,1 stop,In-flight meal not included
3,Multiple carriers,21/05/2019,Delhi,Cochin,DEL → BOM → COK,08:00,21:00,13h,1 stop,No info
4,Air Asia,24/06/2019,Banglore,Delhi,BLR → DEL,23:55,02:45 25 Jun,2h 50m,non-stop,No info


In [11]:
df_train.shape

(10683, 11)

In [12]:
df_test.shape

(2671, 10)

In [14]:
# merge the train and test data into single dataset
df=pd.concat([df_train,df_test])

In [15]:
df.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897.0
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662.0
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882.0
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218.0
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302.0


In [325]:
df.shape

(13354, 11)

In [16]:
#finding the  missing values 
df.isnull().sum()

Airline               0
Date_of_Journey       0
Source                0
Destination           0
Route                 1
Dep_Time              0
Arrival_Time          0
Duration              0
Total_Stops           1
Additional_Info       0
Price              2671
dtype: int64

In [18]:
# which features have the missing values
[features for features in df.columns if df[features].isnull().sum()>0]

['Route', 'Total_Stops', 'Price']

- Price have more missing values because in the test dataset the price columns in target value. when we merge the train and test data. test data price columns are null.

In [20]:
df.duplicated().sum()

246

# Feature Engineering

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 13354 entries, 0 to 2670
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Airline          13354 non-null  object 
 1   Date_of_Journey  13354 non-null  object 
 2   Source           13354 non-null  object 
 3   Destination      13354 non-null  object 
 4   Route            13353 non-null  object 
 5   Dep_Time         13354 non-null  object 
 6   Arrival_Time     13354 non-null  object 
 7   Duration         13354 non-null  object 
 8   Total_Stops      13353 non-null  object 
 9   Additional_Info  13354 non-null  object 
 10  Price            10683 non-null  float64
dtypes: float64(1), object(10)
memory usage: 1.2+ MB


In [22]:
df["Date_of_Journey"]

0       24/03/2019
1        1/05/2019
2        9/06/2019
3       12/05/2019
4       01/03/2019
           ...    
2666     6/06/2019
2667    27/03/2019
2668     6/03/2019
2669     6/03/2019
2670    15/06/2019
Name: Date_of_Journey, Length: 13354, dtype: object

In [23]:
df["Date_of_Journey"].apply(lambda x:x.split("/"))

0       [24, 03, 2019]
1        [1, 05, 2019]
2        [9, 06, 2019]
3       [12, 05, 2019]
4       [01, 03, 2019]
             ...      
2666     [6, 06, 2019]
2667    [27, 03, 2019]
2668     [6, 03, 2019]
2669     [6, 03, 2019]
2670    [15, 06, 2019]
Name: Date_of_Journey, Length: 13354, dtype: object

In [24]:
# split the data of journey columns with "/"
df["Date_of_Journey"].str.split("/")

0       [24, 03, 2019]
1        [1, 05, 2019]
2        [9, 06, 2019]
3       [12, 05, 2019]
4       [01, 03, 2019]
             ...      
2666     [6, 06, 2019]
2667    [27, 03, 2019]
2668     [6, 03, 2019]
2669     [6, 03, 2019]
2670    [15, 06, 2019]
Name: Date_of_Journey, Length: 13354, dtype: object

In [26]:
#first value after split the value is date
df["Date_of_Journey"].str.split("/").str[0]

0       24
1        1
2        9
3       12
4       01
        ..
2666     6
2667    27
2668     6
2669     6
2670    15
Name: Date_of_Journey, Length: 13354, dtype: object

In [27]:
# second value after splitting the value is month
df["Date_of_Journey"].str.split("/").str[1]

0       03
1       05
2       06
3       05
4       03
        ..
2666    06
2667    03
2668    03
2669    03
2670    06
Name: Date_of_Journey, Length: 13354, dtype: object

In [30]:
df["Date"]=df["Date_of_Journey"].apply(lambda x:x.split('/')[0])
df["Month"]=df["Date_of_Journey"].apply(lambda x:x.split('/')[1])
df["Year"]=df["Date_of_Journey"].apply(lambda x:x.split('/')[2])
# Add the new features using the date of journey


In [31]:
df.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price,Date,Month,Year
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897.0,24,03,2019
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662.0,1,05,2019
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882.0,9,06,2019
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218.0,12,05,2019
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302.0,01,03,2019


In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 13354 entries, 0 to 2670
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Airline          13354 non-null  object 
 1   Date_of_Journey  13354 non-null  object 
 2   Source           13354 non-null  object 
 3   Destination      13354 non-null  object 
 4   Route            13353 non-null  object 
 5   Dep_Time         13354 non-null  object 
 6   Arrival_Time     13354 non-null  object 
 7   Duration         13354 non-null  object 
 8   Total_Stops      13353 non-null  object 
 9   Additional_Info  13354 non-null  object 
 10  Price            10683 non-null  float64
 11  Date             13354 non-null  object 
 12  Month            13354 non-null  object 
 13  Year             13354 non-null  object 
dtypes: float64(1), object(13)
memory usage: 1.5+ MB


In [33]:
# convert the object value into int type
df["Date"]=df["Date"].astype(int)
df["Month"]=df["Month"].astype(int)
df["Year"]=df["Year"].astype(int)

In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 13354 entries, 0 to 2670
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Airline          13354 non-null  object 
 1   Date_of_Journey  13354 non-null  object 
 2   Source           13354 non-null  object 
 3   Destination      13354 non-null  object 
 4   Route            13353 non-null  object 
 5   Dep_Time         13354 non-null  object 
 6   Arrival_Time     13354 non-null  object 
 7   Duration         13354 non-null  object 
 8   Total_Stops      13353 non-null  object 
 9   Additional_Info  13354 non-null  object 
 10  Price            10683 non-null  float64
 11  Date             13354 non-null  int64  
 12  Month            13354 non-null  int64  
 13  Year             13354 non-null  int64  
dtypes: float64(1), int64(3), object(10)
memory usage: 1.5+ MB


In [35]:
# delete the data of journey columns
df.drop("Date_of_Journey",axis=1,inplace=True)

In [36]:
df.head()

,Airline,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price,Date,Month,Year
0,IndiGo,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897.0,24,3,2019
1,Air India,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662.0,1,5,2019
2,Jet Airways,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882.0,9,6,2019
3,IndiGo,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218.0,12,5,2019
4,IndiGo,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302.0,1,3,2019


In [37]:
df.Arrival_Time

0       01:10 22 Mar
1              13:15
2       04:25 10 Jun
3              23:30
4              21:35
            ...     
2666    20:25 07 Jun
2667           16:55
2668    04:25 07 Mar
2669           19:15
2670           19:15
Name: Arrival_Time, Length: 13354, dtype: object

In [38]:
df.Arrival_Time.value_counts()

Arrival_Time
19:00           536
21:00           466
19:15           434
16:10           185
12:35           159
               ... 
13:15 22 May      1
08:50 07 Mar      1
06:50 28 Apr      1
00:25 22 Jun      1
07:45 13 May      1
Name: count, Length: 1451, dtype: int64

In [39]:
df["Arrival_Time"].str.split(" ")

0       [01:10, 22, Mar]
1                [13:15]
2       [04:25, 10, Jun]
3                [23:30]
4                [21:35]
              ...       
2666    [20:25, 07, Jun]
2667             [16:55]
2668    [04:25, 07, Mar]
2669             [19:15]
2670             [19:15]
Name: Arrival_Time, Length: 13354, dtype: object

In [40]:
df["Arrival_Time"].str.split(" ").str[0]

0       01:10
1       13:15
2       04:25
3       23:30
4       21:35
        ...  
2666    20:25
2667    16:55
2668    04:25
2669    19:15
2670    19:15
Name: Arrival_Time, Length: 13354, dtype: object

In [41]:
# remove the date in the arrival time
df["Arrival_Time"]=df["Arrival_Time"].apply(lambda x:x.split(" ")[0])


In [42]:
df.head()

,Airline,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price,Date,Month,Year
0,IndiGo,Banglore,New Delhi,BLR → DEL,22:20,01:10,2h 50m,non-stop,No info,3897.0,24,3,2019
1,Air India,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662.0,1,5,2019
2,Jet Airways,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25,19h,2 stops,No info,13882.0,9,6,2019
3,IndiGo,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218.0,12,5,2019
4,IndiGo,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302.0,1,3,2019


In [43]:
df["Arrival_Time"].str.split(":")

0       [01, 10]
1       [13, 15]
2       [04, 25]
3       [23, 30]
4       [21, 35]
          ...   
2666    [20, 25]
2667    [16, 55]
2668    [04, 25]
2669    [19, 15]
2670    [19, 15]
Name: Arrival_Time, Length: 13354, dtype: object

In [44]:
df["Arrival_Time"].str.split(":").str[0]

0       01
1       13
2       04
3       23
4       21
        ..
2666    20
2667    16
2668    04
2669    19
2670    19
Name: Arrival_Time, Length: 13354, dtype: object

In [45]:
# split the arrival time into arrival hour and min
df["Arrival_Hour"]=df["Arrival_Time"].apply(lambda x:x.split(":")[0])
df["Arrival_Min"]=df["Arrival_Time"].apply(lambda x:x.split(":")[1])

In [46]:
df["Arrival_Hour"]=df["Arrival_Hour"].astype(int)
df["Arrival_Min"]=df["Arrival_Min"].astype(int)

In [47]:
df.head()

,Airline,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price,Date,Month,Year,Arrival_Hour,Arrival_Min
0,IndiGo,Banglore,New Delhi,BLR → DEL,22:20,01:10,2h 50m,non-stop,No info,3897.0,24,3,2019,1,10
1,Air India,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662.0,1,5,2019,13,15
2,Jet Airways,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25,19h,2 stops,No info,13882.0,9,6,2019,4,25
3,IndiGo,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218.0,12,5,2019,23,30
4,IndiGo,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302.0,1,3,2019,21,35


In [48]:
df.isnull().sum()

Airline               0
Source                0
Destination           0
Route                 1
Dep_Time              0
Arrival_Time          0
Duration              0
Total_Stops           1
Additional_Info       0
Price              2671
Date                  0
Month                 0
Year                  0
Arrival_Hour          0
Arrival_Min           0
dtype: int64

In [49]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 13354 entries, 0 to 2670
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Airline          13354 non-null  object 
 1   Source           13354 non-null  object 
 2   Destination      13354 non-null  object 
 3   Route            13353 non-null  object 
 4   Dep_Time         13354 non-null  object 
 5   Arrival_Time     13354 non-null  object 
 6   Duration         13354 non-null  object 
 7   Total_Stops      13353 non-null  object 
 8   Additional_Info  13354 non-null  object 
 9   Price            10683 non-null  float64
 10  Date             13354 non-null  int64  
 11  Month            13354 non-null  int64  
 12  Year             13354 non-null  int64  
 13  Arrival_Hour     13354 non-null  int64  
 14  Arrival_Min      13354 non-null  int64  
dtypes: float64(1), int64(5), object(9)
memory usage: 1.6+ MB


In [50]:
#Same as the Departure Time also we change the object format into int
df["Dep_Time"]

0       22:20
1       05:50
2       09:25
3       18:05
4       16:50
        ...  
2666    20:30
2667    14:20
2668    21:50
2669    04:00
2670    04:55
Name: Dep_Time, Length: 13354, dtype: object

In [51]:
df["Dep_Time"].str.split(":").str[0]

0       22
1       05
2       09
3       18
4       16
        ..
2666    20
2667    14
2668    21
2669    04
2670    04
Name: Dep_Time, Length: 13354, dtype: object

In [52]:
df["Dep_Hour"]=df["Dep_Time"].apply(lambda x:x.split(":")[0])
df["Dep_Min"]=df["Dep_Time"].apply(lambda x:x.split(":")[1])

In [53]:
df.head()

,Airline,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price,Date,Month,Year,Arrival_Hour,Arrival_Min,Dep_Hour,Dep_Min
0,IndiGo,Banglore,New Delhi,BLR → DEL,22:20,01:10,2h 50m,non-stop,No info,3897.0,24,3,2019,1,10,22,20
1,Air India,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662.0,1,5,2019,13,15,05,50
2,Jet Airways,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25,19h,2 stops,No info,13882.0,9,6,2019,4,25,09,25
3,IndiGo,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218.0,12,5,2019,23,30,18,05
4,IndiGo,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302.0,1,3,2019,21,35,16,50


In [54]:
df["Dep_Hour"]=df["Dep_Hour"].astype(int)
df["Dep_Min"]=df["Dep_Min"].astype(int)


In [55]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 13354 entries, 0 to 2670
Data columns (total 17 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Airline          13354 non-null  object 
 1   Source           13354 non-null  object 
 2   Destination      13354 non-null  object 
 3   Route            13353 non-null  object 
 4   Dep_Time         13354 non-null  object 
 5   Arrival_Time     13354 non-null  object 
 6   Duration         13354 non-null  object 
 7   Total_Stops      13353 non-null  object 
 8   Additional_Info  13354 non-null  object 
 9   Price            10683 non-null  float64
 10  Date             13354 non-null  int64  
 11  Month            13354 non-null  int64  
 12  Year             13354 non-null  int64  
 13  Arrival_Hour     13354 non-null  int64  
 14  Arrival_Min      13354 non-null  int64  
 15  Dep_Hour         13354 non-null  int64  
 16  Dep_Min          13354 non-null  int64  
dtypes: float64(1), int

In [56]:
df["Route"]

0                   BLR → DEL
1       CCU → IXR → BBI → BLR
2       DEL → LKO → BOM → COK
3             CCU → NAG → BLR
4             BLR → NAG → DEL
                ...          
2666          CCU → DEL → BLR
2667                CCU → BLR
2668          DEL → BOM → COK
2669          DEL → BOM → COK
2670          DEL → BOM → COK
Name: Route, Length: 13354, dtype: object

In [57]:
df["Route"].value_counts()

Route
DEL → BOM → COK                3000
BLR → DEL                      1941
CCU → BOM → BLR                1232
CCU → BLR                       896
BOM → HYD                       785
                               ... 
BOM → JLR → HYD                   1
BOM → NDC → HYD                   1
BOM → COK → MAA → HYD             1
BOM → JDH → JAI → DEL → HYD       1
BOM → IXC → DEL → HYD             1
Name: count, Length: 132, dtype: int64

- Route have different value difficult to change the feature instead of we change the Total stops highly realted to route

In [58]:
df["Total_Stops"].unique()

array(['non-stop', '2 stops', '1 stop', '3 stops', nan, '4 stops'],
      dtype=object)

In [59]:
df["Total_Stops"].value_counts()

Total_Stops
1 stop      7056
non-stop    4340
2 stops     1899
3 stops       56
4 stops        2
Name: count, dtype: int64

In [60]:
#find out the null values and replace the null values
df["Total_Stops"].isnull().sum()

1

In [61]:
#we have only 1 null value, easy to change the value
df[df["Total_Stops"].isnull()]

,Airline,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price,Date,Month,Year,Arrival_Hour,Arrival_Min,Dep_Hour,Dep_Min
9039,Air India,Delhi,Cochin,NaN,09:45,09:25,23h 40m,NaN,No info,7480.0,6,5,2019,9,25,9,45


In [77]:
df[(df['Source'] == "Delhi") & (df['Destination']=='Cochin')].Total_Stops.value_counts()

Total_Stops
1 stop      4015
2 stops     1373
non-stop     262
3 stops       31
Name: count, dtype: int64

In [81]:
df.groupby(["Source","Destination","Total_Stops"]).size().reset_index()

,Source,Destination,Total_Stops,0
0,Banglore,Delhi,non-stop,1582
1,Banglore,New Delhi,1 stop,703
2,Banglore,New Delhi,2 stops,98
3,Banglore,New Delhi,3 stops,8
4,Banglore,New Delhi,4 stops,2
5,Banglore,New Delhi,non-stop,359
6,Chennai,Kolkata,non-stop,456
7,Delhi,Cochin,1 stop,4015
8,Delhi,Cochin,2 stops,1373
9,Delhi,Cochin,3 stops,31


In [82]:
#replace the total_stops values with numbers 
df["Total_Stops"]=df["Total_Stops"].map({'non-stop':0, '2 stops':2, '1 stop':1, '3 stops':3, 'nan':2, '4 stops':4})

In [83]:
df["Total_Stops"].isnull().sum()

1

In [84]:
# replace the missing value in total stop with 2 because from delhi to cochin atleast 2 stops neeed to travel
df["Total_Stops"].fillna(2,inplace=True)

In [85]:
df["Total_Stops"].isnull().sum()

0

In [86]:
df["Total_Stops"].unique()

array([0., 2., 1., 3., 4.])

In [87]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 13354 entries, 0 to 2670
Data columns (total 17 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Airline          13354 non-null  object 
 1   Source           13354 non-null  object 
 2   Destination      13354 non-null  object 
 3   Route            13353 non-null  object 
 4   Dep_Time         13354 non-null  object 
 5   Arrival_Time     13354 non-null  object 
 6   Duration         13354 non-null  object 
 7   Total_Stops      13354 non-null  float64
 8   Additional_Info  13354 non-null  object 
 9   Price            10683 non-null  float64
 10  Date             13354 non-null  int64  
 11  Month            13354 non-null  int64  
 12  Year             13354 non-null  int64  
 13  Arrival_Hour     13354 non-null  int64  
 14  Arrival_Min      13354 non-null  int64  
 15  Dep_Hour         13354 non-null  int64  
 16  Dep_Min          13354 non-null  int64  
dtypes: float64(2), int

In [88]:
df.drop(["Dep_Time","Arrival_Time","Route"],axis=1,inplace=True)

In [89]:
df.head()

,Airline,Source,Destination,Duration,Total_Stops,Additional_Info,Price,Date,Month,Year,Arrival_Hour,Arrival_Min,Dep_Hour,Dep_Min
0,IndiGo,Banglore,New Delhi,2h 50m,0.0,No info,3897.0,24,3,2019,1,10,22,20
1,Air India,Kolkata,Banglore,7h 25m,2.0,No info,7662.0,1,5,2019,13,15,5,50
2,Jet Airways,Delhi,Cochin,19h,2.0,No info,13882.0,9,6,2019,4,25,9,25
3,IndiGo,Kolkata,Banglore,5h 25m,1.0,No info,6218.0,12,5,2019,23,30,18,5
4,IndiGo,Banglore,New Delhi,4h 45m,1.0,No info,13302.0,1,3,2019,21,35,16,50


In [90]:
df["Duration"].str.split(" ")

0        [2h, 50m]
1        [7h, 25m]
2            [19h]
3        [5h, 25m]
4        [4h, 45m]
           ...    
2666    [23h, 55m]
2667     [2h, 35m]
2668     [6h, 35m]
2669    [15h, 15m]
2670    [14h, 20m]
Name: Duration, Length: 13354, dtype: object

In [91]:
df["Duration"].str.split(" ").str[0]

0        2h
1        7h
2       19h
3        5h
4        4h
       ... 
2666    23h
2667     2h
2668     6h
2669    15h
2670    14h
Name: Duration, Length: 13354, dtype: object

In [92]:
df["Duration"].str.split(" ").str[0].str.split('h').str[0]

0        2
1        7
2       19
3        5
4        4
        ..
2666    23
2667     2
2668     6
2669    15
2670    14
Name: Duration, Length: 13354, dtype: object

In [101]:

df["Duration"].str.split(" ").str[1].str.split('m').str[0]

0        50
1        25
2       NaN
3        25
4        45
       ... 
2666     55
2667     35
2668     35
2669     15
2670     20
Name: Duration, Length: 13354, dtype: object

In [102]:
df["Duration"].str.split(" ").str[1].str.split('m').str[0].value_counts()

Duration
30    1818
20    1260
50    1205
45    1154
35    1149
15    1135
55    1121
25    1009
40     803
5      767
10     647
Name: count, dtype: int64

In [103]:
df.head()

,Airline,Source,Destination,Duration,Total_Stops,Additional_Info,Price,Date,Month,Year,Arrival_Hour,Arrival_Min,Dep_Hour,Dep_Min
0,IndiGo,Banglore,New Delhi,2h 50m,0.0,No info,3897.0,24,3,2019,1,10,22,20
1,Air India,Kolkata,Banglore,7h 25m,2.0,No info,7662.0,1,5,2019,13,15,5,50
2,Jet Airways,Delhi,Cochin,19h,2.0,No info,13882.0,9,6,2019,4,25,9,25
3,IndiGo,Kolkata,Banglore,5h 25m,1.0,No info,6218.0,12,5,2019,23,30,18,5
4,IndiGo,Banglore,New Delhi,4h 45m,1.0,No info,13302.0,1,3,2019,21,35,16,50


In [104]:
# df["Duration"].str.split(" ").str[0].str.split('h').str[0]

In [105]:
#above error give one value is durtaion time is 5m not 0 hr 5m 
df[df["Duration"]=="5m"]

,Airline,Source,Destination,Duration,Total_Stops,Additional_Info,Price,Date,Month,Year,Arrival_Hour,Arrival_Min,Dep_Hour,Dep_Min
6474,Air India,Mumbai,Hyderabad,5m,2.0,No info,17327.0,6,3,2019,16,55,16,50
2660,Air India,Mumbai,Hyderabad,5m,2.0,No info,NaN,12,3,2019,16,55,16,50


In [106]:
#it impossible to travel from mumbai to hyd withhin 5 min, we remove the rows
df.drop(6474,axis=0,inplace=True)

In [107]:
df.drop(2660,axis=0,inplace=True)

In [109]:
#convert the duration into hours
df["Duration_Hour"]=df["Duration"].str.split(" ").str[0].str.split('h').str[0].astype(int)*60

In [110]:
df["Duration_min"]=df["Duration"].str.split(" ").str[1].str.split('m').str[0]

In [111]:
df["Duration"].str.split(" ").str[1].str.split('m').str[0].unique()

array(['50', '25', nan, '45', '30', '5', '15', '35', '10', '20', '55',
       '40'], dtype=object)

In [112]:
df.head()

,Airline,Source,Destination,Duration,Total_Stops,Additional_Info,Price,Date,Month,Year,Arrival_Hour,Arrival_Min,Dep_Hour,Dep_Min,Duration_Hour,Duration_min
0,IndiGo,Banglore,New Delhi,2h 50m,0.0,No info,3897.0,24,3,2019,1,10,22,20,120,50
1,Air India,Kolkata,Banglore,7h 25m,2.0,No info,7662.0,1,5,2019,13,15,5,50,420,25
2,Jet Airways,Delhi,Cochin,19h,2.0,No info,13882.0,9,6,2019,4,25,9,25,1140,NaN
3,IndiGo,Kolkata,Banglore,5h 25m,1.0,No info,6218.0,12,5,2019,23,30,18,5,300,25
4,IndiGo,Banglore,New Delhi,4h 45m,1.0,No info,13302.0,1,3,2019,21,35,16,50,240,45


In [114]:
def convert_duration_to_minutes(duration_str):
    hours, minutes = duration_str.split('h ')
    minutes = minutes.replace('min', '')
    total_minutes = int(hours) * 60 + int(minutes)
    return total_minutes

In [115]:
df.isnull().sum()

Airline               0
Source                0
Destination           0
Duration              0
Total_Stops           0
Additional_Info       0
Price              2670
Date                  0
Month                 0
Year                  0
Arrival_Hour          0
Arrival_Min           0
Dep_Hour              0
Dep_Min               0
Duration_Hour         0
Duration_min       1283
dtype: int64

In [116]:
df["Duration_min"].fillna(0,inplace=True)

In [117]:
df["Duration_min"].isnull().sum()

0

In [118]:
df["Duration_min"]=df["Duration_min"].astype(int)

In [119]:
#combine the both hours and min into duration with total mins
df["Duration"]=df["Duration_Hour"]+df["Duration_min"]


In [120]:
df.head()

,Airline,Source,Destination,Duration,Total_Stops,Additional_Info,Price,Date,Month,Year,Arrival_Hour,Arrival_Min,Dep_Hour,Dep_Min,Duration_Hour,Duration_min
0,IndiGo,Banglore,New Delhi,170,0.0,No info,3897.0,24,3,2019,1,10,22,20,120,50
1,Air India,Kolkata,Banglore,445,2.0,No info,7662.0,1,5,2019,13,15,5,50,420,25
2,Jet Airways,Delhi,Cochin,1140,2.0,No info,13882.0,9,6,2019,4,25,9,25,1140,0
3,IndiGo,Kolkata,Banglore,325,1.0,No info,6218.0,12,5,2019,23,30,18,5,300,25
4,IndiGo,Banglore,New Delhi,285,1.0,No info,13302.0,1,3,2019,21,35,16,50,240,45


In [121]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 13351 entries, 0 to 2670
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Airline          13351 non-null  object 
 1   Source           13351 non-null  object 
 2   Destination      13351 non-null  object 
 3   Duration         13351 non-null  int64  
 4   Total_Stops      13351 non-null  float64
 5   Additional_Info  13351 non-null  object 
 6   Price            10681 non-null  float64
 7   Date             13351 non-null  int64  
 8   Month            13351 non-null  int64  
 9   Year             13351 non-null  int64  
 10  Arrival_Hour     13351 non-null  int64  
 11  Arrival_Min      13351 non-null  int64  
 12  Dep_Hour         13351 non-null  int64  
 13  Dep_Min          13351 non-null  int64  
 14  Duration_Hour    13351 non-null  int64  
 15  Duration_min     13351 non-null  int64  
dtypes: float64(2), int64(10), object(4)
memory usage: 1.7+ MB


In [122]:
from sklearn.preprocessing import LabelEncoder
label=LabelEncoder()

In [124]:
df["Airline"].value_counts()

Airline
Jet Airways                          4745
IndiGo                               2564
Air India                            2190
Multiple carriers                    1543
SpiceJet                             1026
Vistara                               608
Air Asia                              405
GoAir                                 240
Multiple carriers Premium economy      16
Jet Airways Business                    8
Vistara Premium economy                 5
Trujet                                  1
Name: count, dtype: int64

In [125]:
#convert the airline values into the numerical using the Label Encoder
df["Airline"]=label.fit_transform(df["Airline"])

In [126]:
df.head()

,Airline,Source,Destination,Duration,Total_Stops,Additional_Info,Price,Date,Month,Year,Arrival_Hour,Arrival_Min,Dep_Hour,Dep_Min,Duration_Hour,Duration_min
0,3,Banglore,New Delhi,170,0.0,No info,3897.0,24,3,2019,1,10,22,20,120,50
1,1,Kolkata,Banglore,445,2.0,No info,7662.0,1,5,2019,13,15,5,50,420,25
2,4,Delhi,Cochin,1140,2.0,No info,13882.0,9,6,2019,4,25,9,25,1140,0
3,3,Kolkata,Banglore,325,1.0,No info,6218.0,12,5,2019,23,30,18,5,300,25
4,3,Banglore,New Delhi,285,1.0,No info,13302.0,1,3,2019,21,35,16,50,240,45


In [127]:
# convert the all categorical values into numerical values using the label encoder
df["Destination"]=label.fit_transform(df["Destination"])
df["Source"]=label.fit_transform(df["Source"])
df["Additional_Info"]=label.fit_transform(df["Additional_Info"])

In [128]:
df.head()

,Airline,Source,Destination,Duration,Total_Stops,Additional_Info,Price,Date,Month,Year,Arrival_Hour,Arrival_Min,Dep_Hour,Dep_Min,Duration_Hour,Duration_min
0,3,0,5,170,0.0,8,3897.0,24,3,2019,1,10,22,20,120,50
1,1,3,0,445,2.0,8,7662.0,1,5,2019,13,15,5,50,420,25
2,4,2,1,1140,2.0,8,13882.0,9,6,2019,4,25,9,25,1140,0
3,3,3,0,325,1.0,8,6218.0,12,5,2019,23,30,18,5,300,25
4,3,0,5,285,1.0,8,13302.0,1,3,2019,21,35,16,50,240,45


In [129]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 13351 entries, 0 to 2670
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Airline          13351 non-null  int64  
 1   Source           13351 non-null  int64  
 2   Destination      13351 non-null  int64  
 3   Duration         13351 non-null  int64  
 4   Total_Stops      13351 non-null  float64
 5   Additional_Info  13351 non-null  int64  
 6   Price            10681 non-null  float64
 7   Date             13351 non-null  int64  
 8   Month            13351 non-null  int64  
 9   Year             13351 non-null  int64  
 10  Arrival_Hour     13351 non-null  int64  
 11  Arrival_Min      13351 non-null  int64  
 12  Dep_Hour         13351 non-null  int64  
 13  Dep_Min          13351 non-null  int64  
 14  Duration_Hour    13351 non-null  int64  
 15  Duration_min     13351 non-null  int64  
dtypes: float64(2), int64(14)
memory usage: 1.7 MB


In [130]:
df.isnull().sum()

Airline               0
Source                0
Destination           0
Duration              0
Total_Stops           0
Additional_Info       0
Price              2670
Date                  0
Month                 0
Year                  0
Arrival_Hour          0
Arrival_Min           0
Dep_Hour              0
Dep_Min               0
Duration_Hour         0
Duration_min          0
dtype: int64

In [133]:
df.duplicated().sum()

249

In [134]:
df.drop_duplicates()

,Airline,Source,Destination,Duration,Total_Stops,Additional_Info,Price,Date,Month,Year,Arrival_Hour,Arrival_Min,Dep_Hour,Dep_Min,Duration_Hour,Duration_min
0,3,0,5,170,0.0,8,3897.0,24,3,2019,1,10,22,20,120,50
1,1,3,0,445,2.0,8,7662.0,1,5,2019,13,15,5,50,420,25
2,4,2,1,1140,2.0,8,13882.0,9,6,2019,4,25,9,25,1140,0
3,3,3,0,325,1.0,8,6218.0,12,5,2019,23,30,18,5,300,25
4,3,0,5,285,1.0,8,13302.0,1,3,2019,21,35,16,50,240,45
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2666,1,3,0,1435,1.0,8,NaN,6,6,2019,20,25,20,30,1380,55
2667,3,3,0,155,0.0,8,NaN,27,3,2019,16,55,14,20,120,35
2668,4,2,1,395,1.0,8,NaN,6,3,2019,4,25,21,50,360,35
2669,1,2,1,915,1.0,8,NaN,6,3,2019,19,15,4,0,900,15


## Conclusions - Observations
- Perfrom Feature Engineering techniques with Flight Price Prediction Dataset.
- Create new features date,month, year of journey using the Date of journey column.
- Create new features arrival hour and min using arrivatl time ans Departure time also.
- Convert the Total Stops with numerical values in how many stops
- Convert the duration column in timestamp into mintues
- Convert the all Categorical values into numerical Values using the Lable Encoder